In [1]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd
import numpy as np
import string
from IPython.display import display, HTML

In [2]:
def del_quote(elems):
    elem_list = []
    for elem in elems:
        elem = string.replace(elem, "'"," ")
        elem_list.append(elem)
    return elem_list 

def get_text(elems):
    elems_list = []
    for elem in elems:
        elem = elem.text
        elem = str(elem)
        elems_list.append(elem)
    return elems_list

def regularize(elems):
    elem_list = []
    for elem in elems:
        # to aviod bug from quotes
        elem = string.replace(elem, "'"," ")
        elem = re.findall('[a-zA-Z0-9\.,: ]+', elem)
        elem = map(str,elem)
        elem_list.append(elem)
    return elem_list 

def get_list(elems, attr_name):
    elem_list = []
    for elem in elems:
        elem = elem.get_attribute(attr_name)
        elem = str(elem)
        elem_list.append(elem)
    return elem_list 


In [3]:
def get_champrank(summoner):

    # open web browser
    driver = webdriver.Chrome('/Users/Sarine/Desktop/chromedriver')
    driver.set_window_size(1400,1000)
    # go to website

    url = 'http://na.op.gg/summoner/champions/userName={}'.format(summoner)
    driver.get(url)
    
    refresh = driver.find_element_by_id("SummonerRefreshButton")
    refresh.click()
    
    time.sleep(5)

    click_normal = driver.find_element_by_xpath("//li[contains(@data-tab-show-class,'normal')]")
    click_normal.click()
    
    time.sleep(5)
    
    rank_number = driver.find_elements_by_xpath("//div[contains(@class,'normal')]/table[@class='ChampionStatsTable sortable tablesorter tablesorter-default']/tbody[@class='Body']/tr[contains(@class,'Row')]/td[@class='Rank Cell']")
    rank_champ = driver.find_elements_by_xpath("//div[contains(@class,'normal')]/table[@class='ChampionStatsTable sortable tablesorter tablesorter-default']/tbody[@class='Body']/tr[contains(@class,'Row')]/td[@class='ChampionName Cell']")
    
    rank_number = pd.DataFrame(get_text(rank_number))
    
    rank_champ = get_text(rank_champ)
    
    rank_champ = del_quote(rank_champ)
    rank_champ = pd.DataFrame(rank_champ)

    rank = pd.concat([rank_number,rank_champ], axis=1)
    rank.columns = ['top','champ']
    
    rank['top'] = rank['top'].astype(float)
    rank['top'] = rank['top']/rank.shape[0]
    
    driver.close()
    
    return rank

In [4]:
def get_gameinfo(summoner):

    # functions

    # open web browser
    driver = webdriver.Chrome('/Users/Sarine/Desktop/chromedriver')
    driver.set_window_size(1400,1000)
    # go to website
    driver.get("http://na.op.gg/")
    # print driver.current_url
    # elem = driver.find_element_by_id('searchDiv_2')
    elem = driver.find_element_by_name('userName')
    elem.clear()
    elem.send_keys(summoner)
    elem.send_keys(Keys.RETURN)
    
    print('Summoner %s Transportation Successed!') %summoner
    
    # fresh to get the latest data
    refresh = driver.find_element_by_id("SummonerRefreshButton")
    refresh.click()
    
    time.sleep(20)

    print('Memories Extracting...')

    # choose normal game
    select = driver.find_element_by_xpath("//option[@value='normal']")
    select.click()

    # to show all records

    time.sleep(10)
    
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            load_more = driver.find_element_by_xpath("//div[@class='GameMoreButton Box']")
            load_more.click()
            time.sleep(1.5)
        except:
            break

    print('Memories Extracted! Going Through Battle Records...')            
            
#     to show game detail
    
    manually_import = driver.find_elements_by_xpath("//div[@class='AlertMessageButton']/button[@class='Button SemiRound Green']")
    
    driver.execute_script("window.scrollTo(0, 0);")
    
    for manually in manually_import: 
        while True:
            try:
                manually.location_once_scrolled_into_view  
                manually.click()
                time.sleep(5)
                break
            except:
                continue   
                
    driver.execute_script("window.scrollTo(0, 0);")

    details = driver.find_elements_by_xpath("//button[@class='Button MatchDetail']")
    # details

    for detail in details: 
        while True:
            try:
                detail.location_once_scrolled_into_view  
                detail.click()
                time.sleep(0.1)
                break
            except:
                continue

    print('Extracting Personal Performace...')        
        
    summoner = driver.find_elements_by_xpath("//tr[contains(@class,'isRequester')]")
    
    if len(summoner) < 30:
        time.sleep(60)
        
    summoner = driver.find_elements_by_xpath("//tr[contains(@class,'isRequester')]")
    
    df = pd.DataFrame(regularize(get_text(summoner)))
    
    print('Performace Flow Forged!')
    
    while df.shape[1] == 18:
        try:
            del df[17]
        except:
            break
            
    df[16] = df[16].map(lambda x:x.split(' ')[1:]).map(lambda x:' '.join(x))

    print('Go Analyzing...')  
    
    # add time into
    game_length = driver.find_elements_by_xpath("//div[@class='GameLength']")
    game_result = driver.find_elements_by_xpath("//div[@class='GameResult']")
    game_timestamp = driver.find_elements_by_xpath("//div[@class='TimeStamp']/span")

    length = pd.DataFrame(get_text(game_length))
    result = pd.DataFrame(get_text(game_result))
    timestamp = pd.DataFrame(get_list(game_timestamp,'title'))
    df = pd.concat([df, length, result, timestamp], axis=1)
    
    cols_of_interest = [0,2,4,5,6,7,11,12,14,16]
    df = df[cols_of_interest]
     
    df.columns = ['champ','length','result','time','name',
                'kill','death','assist','partcipate','ward_place',
                'ward_kill','cs','tier']
    
    df.loc[:,'time'] = pd.to_datetime(df.loc[:,'time'])
    
    df['tier'].replace(to_replace = ['Unranked','Level 30','Level 29','Level 28','Level 27','Level 26','Level 25','Level 24','Level 23','Level 22','Level 21','Level 20','Level 19','Level 18','Level 17','Level 16','Level 15','Level 14','Level 13','Level 12','Level 11','Level 10','Level 9','Level 8','Level 7','Level 6','Level 5','Level 4','Level 3','Level 2','Level 1','Bronze 5','Bronze 4','Bronze 3','Bronze 2','Bronze 1','Silver 5','Silver 4','Silver 3','Silver 2','Silver 1','Gold 5','Gold 4','Gold 3','Gold 2','Gold 1','Platinum 5','Platinum 4','Platinum 3','Platinum 2','Platinum 1','Diamond 5','Diamond 4','Diamond 3','Diamond 2','Diamond 1','Master','Challenger'],
                         value = ['8','30','30','30','30','30','30','30','30','30','30','20','20','20','20','20','20','20','20','20','20','10','9','8','7','6','5','4','3','2','1','20','24','26','28','30','32','34','36','38','40','44','48','52','56','60','66','72','78','84','90','98','106','114','122','130','140','200'], inplace= True)
    
    df['kill'] = df['kill'].astype(float)
    df['death'] = df['death'].astype(float)
    df['assist'] = df['assist'].astype(float)
    df['partcipate'] = df['partcipate'].astype(float)
    df['ward_place'] = df['ward_place'].astype(float)
    df['ward_kill'] = df['ward_kill'].astype(float)
    df['cs'] = df['cs'].astype(float)
    df['tier'] = df['tier'].astype(float)
    
    print('Clone Made!!')  
    driver.close()
    
    return df

In [5]:
def get_table(summoner, champ):


####################     gameinfo    ############################################  

    # open web browser
    driver = webdriver.Chrome('/Users/Sarine/Desktop/chromedriver')
    driver.set_window_size(1400,1000)
    # go to website
    driver.get("http://na.op.gg/")
    # print driver.current_url
    # elem = driver.find_element_by_id('searchDiv_2')
    elem = driver.find_element_by_name('userName')
    elem.clear()
    elem.send_keys(summoner)
    elem.send_keys(Keys.RETURN)
    
    # fresh to get the latest data
    refresh = driver.find_element_by_id("SummonerRefreshButton")
    refresh.click()
    
    time.sleep(20)

    # choose normal game
    select = driver.find_element_by_xpath("//option[@value='normal']")
    select.click()

    # to show all records

    time.sleep(10)
    
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            load_more = driver.find_element_by_xpath("//div[@class='GameMoreButton Box']")
            load_more.click()
            time.sleep(1.5)
        except:
            break
            
    # to show game detail
    
    manually_import = driver.find_elements_by_xpath("//div[@class='AlertMessageButton']/button[@class='Button SemiRound Green']")
    
    driver.execute_script("window.scrollTo(0, 0);")
    
    for manually in manually_import: 
        while True:
            try:
                manually.location_once_scrolled_into_view  
                manually.click()
                time.sleep(5)
                break
            except:
                continue   
                
    driver.execute_script("window.scrollTo(0, 0);")

    details = driver.find_elements_by_xpath("//button[@class='Button MatchDetail']")
    # details

    for detail in details: 
        while True:
            try:
                detail.location_once_scrolled_into_view  
                detail.click()
                time.sleep(0.1)
                break
            except:
                continue

    time.sleep(15)
        
    summoner_list = driver.find_elements_by_xpath("//tr[contains(@class,'isRequester')]")
    
    if len(summoner_list) < 30:
        time.sleep(60)
        
    summoner_list = driver.find_elements_by_xpath("//tr[contains(@class,'isRequester')]")
    
    df = pd.DataFrame(regularize(get_text(summoner_list)))
    
    time.sleep(5)
   
    while df.shape[1] == 18:
        try:
            del df[17]
        except:
            break
            
    df[16] = df[16].map(lambda x:x.split(' ')[1:]).map(lambda x:' '.join(x))

    # add time into
    game_length = driver.find_elements_by_xpath("//div[@class='GameLength']")
    game_result = driver.find_elements_by_xpath("//div[@class='GameResult']")
    game_timestamp = driver.find_elements_by_xpath("//div[@class='TimeStamp']/span")

    length = pd.DataFrame(get_text(game_length))
    result = pd.DataFrame(get_text(game_result))
    timestamp = pd.DataFrame(get_list(game_timestamp,'title'))
    df = pd.concat([df, length, result, timestamp], axis=1)
    
    cols_of_interest = [0,2,4,5,6,7,11,12,14,16]
    df = df[cols_of_interest]
     
    df.columns = ['champ','length','result','time','name',
                'kill','death','assist','partcipate','ward_place',
                'ward_kill','cs','tier']
    
    df.loc[:,'time'] = pd.to_datetime(df.loc[:,'time'])
    
    df['tier'].replace(to_replace = ['Unranked','Level 30','Level 29','Level 28','Level 27','Level 26','Level 25','Level 24','Level 23','Level 22','Level 21','Level 20','Level 19','Level 18','Level 17','Level 16','Level 15','Level 14','Level 13','Level 12','Level 11','Level 10','Level 9','Level 8','Level 7','Level 6','Level 5','Level 4','Level 3','Level 2','Level 1','Bronze 5','Bronze 4','Bronze 3','Bronze 2','Bronze 1','Silver 5','Silver 4','Silver 3','Silver 2','Silver 1','Gold 5','Gold 4','Gold 3','Gold 2','Gold 1','Platinum 5','Platinum 4','Platinum 3','Platinum 2','Platinum 1','Diamond 5','Diamond 4','Diamond 3','Diamond 2','Diamond 1','Master','Challenger'],
                         value = ['8','30','30','30','30','30','30','30','30','30','30','20','20','20','20','20','20','20','20','20','20','10','9','8','7','6','5','4','3','2','1','20','24','26','28','30','32','34','36','38','40','44','48','52','56','60','66','72','78','84','90','98','106','114','122','130','140','200'], inplace= True)
    
    df['kill'] = df['kill'].astype(float)
    df['death'] = df['death'].astype(float)
    df['assist'] = df['assist'].astype(float)
    df['partcipate'] = df['partcipate'].astype(float)
    df['ward_place'] = df['ward_place'].astype(float)
    df['ward_kill'] = df['ward_kill'].astype(float)
    df['cs'] = df['cs'].astype(float)
    df['tier'] = df['tier'].astype(float)
    
####################     gameinfo    ############################################    
    
    
####################     champrank    ###########################################    

    url = 'http://na.op.gg/summoner/champions/userName={}'.format(summoner)
    driver.get(url)
    
#     refresh = driver.find_element_by_id("SummonerRefreshButton")
#     refresh.click()
    
    time.sleep(5)

    click_normal = driver.find_element_by_xpath("//li[contains(@data-tab-show-class,'normal')]")
    click_normal.click()
    
    time.sleep(5)
    
    rank_number = driver.find_elements_by_xpath("//div[contains(@class,'normal')]/table[@class='ChampionStatsTable sortable tablesorter tablesorter-default']/tbody[@class='Body']/tr[contains(@class,'Row')]/td[@class='Rank Cell']")
    rank_champ = driver.find_elements_by_xpath("//div[contains(@class,'normal')]/table[@class='ChampionStatsTable sortable tablesorter tablesorter-default']/tbody[@class='Body']/tr[contains(@class,'Row')]/td[@class='ChampionName Cell']")
    
    rank_number = pd.DataFrame(get_text(rank_number))
    
    rank_champ = get_text(rank_champ)
    
    rank_champ = del_quote(rank_champ)
    rank_champ = pd.DataFrame(rank_champ)

    rank = pd.concat([rank_number,rank_champ], axis=1)
    rank.columns = ['top','champ']
    
    rank['top'] = rank['top'].astype(float)
    rank['top'] = rank['top']/rank.shape[0]
    
####################     champrank    ###########################################     

####################     get table    ###########################################  

    gameinfo = df
    champrank = rank
    
    not_enough = gameinfo.mean()

    champ_games = gameinfo.loc[(gameinfo.champ == champ)]
    champ_rank = champrank.loc[(champrank.champ == champ)]
    
    win_number = gameinfo.loc[(gameinfo.result == "Victory")].shape[0]
    whole_number = gameinfo.shape[0]
    win_rate = float(win_number)/float(whole_number)

    champ_win_number = champ_games.loc[(champ_games.result == "Victory")].shape[0]
    champ_whole_number = champ_games.shape[0]
    champ_win_rate = float(champ_win_number)/float(champ_whole_number)
        
    
    champ_games_mean = champ_games.groupby(champ_games.champ).mean()

    if champ_games.shape[0] < 4:
        champ_table = pd.DataFrame([[gameinfo.shape[0],champ_games.shape[0],win_rate,(not_enough.kill+not_enough.assist)/(not_enough.death+1), not_enough.partcipate, not_enough.ward_place, not_enough.ward_kill, not_enough.cs, not_enough.tier, champ_rank.iloc[0,0]]]
                      , columns = ['game_frequency','champ_frequency','win_rate', 'kda','partcipate','ward_place','ward_kill','cs','tier','top'])
    else:
        champ_table = pd.DataFrame([[gameinfo.shape[0],champ_games.shape[0], champ_win_rate,(champ_games_mean.kill[0]+champ_games_mean.assist[0])/(champ_games_mean.death[0]+1), champ_games_mean.partcipate[0], champ_games_mean.ward_place[0], champ_games_mean.ward_kill[0], champ_games_mean.cs[0], champ_games_mean.tier[0], champ_rank.iloc[0,0]]]
                      , columns = ['game_frequency','champ_frequency','win_rate', 'kda','partcipate','ward_place','ward_kill','cs','tier','top'])

####################     get table    ###########################################  

    driver.close()
    return champ_table

In [6]:
def get_match(summoner):

####################     gameinfo    ############################################  

    # open web browser
    driver = webdriver.Chrome('/Users/Sarine/Desktop/chromedriver')
    driver.set_window_size(1400,1000)
    # go to website
    driver.get("http://na.op.gg/")
    # print driver.current_url
    # elem = driver.find_element_by_id('searchDiv_2')
    elem = driver.find_element_by_name('userName')
    elem.clear()
    elem.send_keys(summoner)
    elem.send_keys(Keys.RETURN)
    
    # fresh to get the latest data
    refresh = driver.find_element_by_id("SummonerRefreshButton")
    refresh.click()
    
    time.sleep(20)

    # choose normal game
    select = driver.find_element_by_xpath("//option[@value='normal']")
    select.click()

    # to show all records

    time.sleep(10)
    
    while True:
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            load_more = driver.find_element_by_xpath("//div[@class='GameMoreButton Box']")
            load_more.click()
            time.sleep(1.5)
        except:
            break        
            
    
    driver.execute_script("window.scrollTo(0, 0);")
    
    manually_import = driver.find_elements_by_xpath("//div[@class='AlertMessageButton']/button[@class='Button SemiRound Green']")
    
    for manually in manually_import: 
        while True:
            try:
                manually.location_once_scrolled_into_view  
                manually.click()
                time.sleep(5)
                break
            except:
                continue    
    
    # to show game detail
    
    driver.execute_script("window.scrollTo(0, 0);")
    
    details = driver.find_elements_by_xpath("//button[@class='Button MatchDetail']")
    # details
 

    for detail in details: 
        while True:
            try:
                detail.location_once_scrolled_into_view  
                detail.click()
                time.sleep(0.1)
                break
            except:
                continue
        
        
    summoner_list = driver.find_elements_by_xpath("//tr[contains(@class,'isRequester')]")
    
    print('Summoner %s Transportation Successed!') %summoner
    
    if len(summoner_list) < 30:
        time.sleep(60)
    
    that_game = driver.find_elements_by_xpath("//div[@class='Team']")
    that_game_result = str(driver.find_element_by_xpath("//div[@class='GameResult']").text)
    
    that_game_1 = string.replace(that_game[0].text, "'"," ")
    team_1_info = map(str,(re.findall('[a-zA-Z0-9\.,: ]+', that_game_1)))
    
    team_1_info_df = pd.DataFrame(team_1_info).transpose()
    display(HTML(team_1_info_df.to_html(index=False)))
    
    that_game_2 = string.replace(that_game[1].text, "'"," ")
    team_2_info = map(str,(re.findall('[a-zA-Z0-9\.,: ]+', that_game_2)))
    
    team_2_info_df = pd.DataFrame(team_2_info).transpose()
    display(HTML(team_2_info_df.to_html(index=False)))

    for n in range(0,5):
        if team_1_info[2*n+1].lower() == summoner:
            champ = team_1_info[2*n]    

    for n in range(0,5):
        if team_2_info[2*n+1].lower() == summoner:
            champ = team_2_info[2*n]
    
    time.sleep(15)
    
    summoner_list = driver.find_elements_by_xpath("//tr[contains(@class,'isRequester')]")
    
    df = pd.DataFrame(regularize(get_text(summoner_list)))
    display(HTML(df.to_html(index=False)))

    
    print('Performace Extrated!')
    
    while df.shape[1] == 18:
        try:
            del df[17]
        except:
            break
            
    df[16] = df[16].map(lambda x:x.split(' ')[1:]).map(lambda x:' '.join(x))
    
    # add time into
    game_length = driver.find_elements_by_xpath("//div[@class='GameLength']")
    game_result = driver.find_elements_by_xpath("//div[@class='GameResult']")
    game_timestamp = driver.find_elements_by_xpath("//div[@class='TimeStamp']/span")

    length = pd.DataFrame(get_text(game_length))
    result = pd.DataFrame(get_text(game_result))
    timestamp = pd.DataFrame(get_list(game_timestamp,'title'))
    df = pd.concat([df, length, result, timestamp], axis=1)
    
    cols_of_interest = [0,2,4,5,6,7,11,12,14,16]
    df = df[cols_of_interest]
     
    df.columns = ['champ','length','result','time','name',
                'kill','death','assist','partcipate','ward_place',
                'ward_kill','cs','tier']
    
    df.loc[:,'time'] = pd.to_datetime(df.loc[:,'time'])
    
    df['tier'].replace(to_replace = ['Unranked','Level 30','Level 29','Level 28','Level 27','Level 26','Level 25','Level 24','Level 23','Level 22','Level 21','Level 20','Level 19','Level 18','Level 17','Level 16','Level 15','Level 14','Level 13','Level 12','Level 11','Level 10','Level 9','Level 8','Level 7','Level 6','Level 5','Level 4','Level 3','Level 2','Level 1','Bronze 5','Bronze 4','Bronze 3','Bronze 2','Bronze 1','Silver 5','Silver 4','Silver 3','Silver 2','Silver 1','Gold 5','Gold 4','Gold 3','Gold 2','Gold 1','Platinum 5','Platinum 4','Platinum 3','Platinum 2','Platinum 1','Diamond 5','Diamond 4','Diamond 3','Diamond 2','Diamond 1','Master','Challenger'],
                         value = ['8','30','30','30','30','30','30','30','30','30','30','20','20','20','20','20','20','20','20','20','20','10','9','8','7','6','5','4','3','2','1','20','24','26','28','30','32','34','36','38','40','44','48','52','56','60','66','72','78','84','90','98','106','114','122','130','140','200'], inplace= True)
    
    
    df['kill'] = df['kill'].astype(float)
    df['death'] = df['death'].astype(float)
    df['assist'] = df['assist'].astype(float)
    df['partcipate'] = df['partcipate'].astype(float)
    df['ward_place'] = df['ward_place'].astype(float)
    df['ward_kill'] = df['ward_kill'].astype(float)
    df['cs'] = df['cs'].astype(float)
    df['tier'] = df['tier'].astype(float)
    
    print('Clone Made!!')   

####################     gameinfo    ############################################    
    
    
####################     champrank    ###########################################    

    url = 'http://na.op.gg/summoner/champions/userName={}'.format(summoner)
    driver.get(url)
    
#     refresh = driver.find_element_by_id("SummonerRefreshButton")
#     refresh.click()
    
    time.sleep(5)

    click_normal = driver.find_element_by_xpath("//li[contains(@data-tab-show-class,'normal')]")
    click_normal.click()
    
    time.sleep(5)
    
    rank_number = driver.find_elements_by_xpath("//div[contains(@class,'normal')]/table[@class='ChampionStatsTable sortable tablesorter tablesorter-default']/tbody[@class='Body']/tr[contains(@class,'Row')]/td[@class='Rank Cell']")
    rank_champ = driver.find_elements_by_xpath("//div[contains(@class,'normal')]/table[@class='ChampionStatsTable sortable tablesorter tablesorter-default']/tbody[@class='Body']/tr[contains(@class,'Row')]/td[@class='ChampionName Cell']")
        
    rank_number = pd.DataFrame(get_text(rank_number))
    
    rank_champ = get_text(rank_champ)
    
    rank_champ = del_quote(rank_champ)
    rank_champ = pd.DataFrame(rank_champ)

    rank = pd.concat([rank_number,rank_champ], axis=1)
    rank.columns = ['top','champ']
    
    rank['top'] = rank['top'].astype(float)
    rank['top'] = rank['top']/rank.shape[0]
    
####################     champrank    ###########################################     

####################     get table    ###########################################  

    gameinfo = df
    
    champrank = rank
    
    not_enough = gameinfo.mean()

    champ_games = gameinfo.loc[(gameinfo.champ == champ)]
    champ_rank = champrank.loc[(champrank.champ == champ)]
    
    win_number = gameinfo.loc[(gameinfo.result == "Victory")].shape[0]
    whole_number = gameinfo.shape[0]
    win_rate = float(win_number)/float(whole_number)

    champ_win_number = champ_games.loc[(champ_games.result == "Victory")].shape[0]
    champ_whole_number = champ_games.shape[0]
    champ_win_rate = float(champ_win_number)/float(champ_whole_number)
        
    champ_games_mean = champ_games.groupby(champ_games.champ).mean()
    
    if champ_games.shape[0] < 4:
        champ_table = pd.DataFrame([[gameinfo.shape[0],champ_games.shape[0],win_rate,(not_enough.kill+not_enough.assist)/(not_enough.death+1), not_enough.partcipate, not_enough.ward_place, not_enough.ward_kill, not_enough.cs, not_enough.tier, champ_rank.iloc[0,0]]]
                      , columns = ['game_frequency','champ_frequency','win_rate', 'kda','partcipate','ward_place','ward_kill','cs','tier','top'])
    else:
        champ_table = pd.DataFrame([[gameinfo.shape[0],champ_games.shape[0],champ_win_rate,(champ_games_mean.kill[0]+champ_games_mean.assist[0])/(champ_games_mean.death[0]+1), champ_games_mean.partcipate[0], champ_games_mean.ward_place[0], champ_games_mean.ward_kill[0], champ_games_mean.cs[0], champ_games_mean.tier[0], champ_rank.iloc[0,0]]]
                      , columns = ['game_frequency','champ_frequency','win_rate', 'kda','partcipate','ward_place','ward_kill','cs','tier','top'])
    
    print("#####################   Table of %s's %s  #####################") % (summoner,champ)
    display(HTML(champ_table.to_html(index=False)))
    
####################     get table    ###########################################  
    
    team_1 = pd.DataFrame()

    for n in range(0,5):
        if team_1_info[2*n+1].lower() == summoner:
            print('Summoner in T1')
        else:
            print("#####################   Table of %s's %s  #####################") % (team_1_info[2*n+1],team_1_info[2*n])            
            while True:
                try:
                    team_1_member = get_table(team_1_info[2*n+1],team_1_info[2*n])
                    display(HTML(team_1_member.to_html(index=False)))
                    team_1 = team_1.append(team_1_member)
                    break
                except:
                    continue
    
    
    
    team_2 = pd.DataFrame()

    for n in range(0,5):
        if team_2_info[2*n+1].lower() == summoner:
            print('Summoner in T2')
        else:
            print("#####################   Table of %s's %s  #####################") % (team_2_info[2*n+1],team_2_info[2*n])
            while True:
                try:
                    team_2_member = get_table(team_2_info[2*n+1],team_2_info[2*n])
                    display(HTML(team_2_member.to_html(index=False)))
                    team_2 = team_2.append(team_2_member)
                    break
                except:
                    continue
            
            
    if team_1.shape[0] == 4:
        team_1 = team_1.append(champ_table)
        in_team_1 = 1
    else:
        team_2 = team_2.append(champ_table)
        in_team_1 = -1
    
    if that_game_result == 'Victory':
        
        summoner_win = 1
    else:
        summoner_win = -1
        

    match_result_token = summoner_win*in_team_1
        
        
    team_1_overall = pd.DataFrame(team_1.mean())

    team_1_overall = team_1_overall.transpose()
    
    team_1_overall.columns = ['game_frequency_1','champ_frequency_1','champ_win_rate_1','kda_1','partcipate_1','ward_place_1','ward_kill_1','cs_1','tier_1','top_1']
        
    team_2_overall = pd.DataFrame(team_2.mean())

    team_2_overall = team_2_overall.transpose()   
    
    team_2_overall.columns = ['game_frequency_2','champ_frequency_2','champ_win_rate_2','kda_2','partcipate_2','ward_place_2','ward_kill_2','cs_2','tier_2','top_2']
    
    match_result = pd.DataFrame([[match_result_token]], columns=['match_result'])
    
    match_info = pd.concat([team_1_overall,team_2_overall,match_result], axis= 1)
    
    display(HTML(match_info.to_html(index=False)))
    
    driver.close()
    
    return match_info

In [7]:
summoners = ['underpaidsniper','allalongwithyou','jezterking','bucksatan1','papacream','gashua','chienziillaaaa','graycastagnier','wbztr','2headedduck']

In [8]:
# while True:
#     for summoner in summoners:
#         try:
#             f = open('match_data', 'a')
#             match = get_match(summoner)  
#             match.to_csv('%s' % summoners[0], sep=',', encoding='utf-8')
#             match.to_csv(f, header = False)
#             f.close()
#             break
#         except:
#             continue

In [ ]:
while True:
    try:
        f = open('match_data', 'a')
        match = get_match(summoners[0])  
        match.to_csv('%s' % summoners[0], sep=',', encoding='utf-8')
        match.to_csv(f, header = False)
        f.close()
        del summoners[0]
    except:
        continue

Summoner underpaidsniper Transportation Successed!


0,1,2,3,4,5,6,7,8,9
Twisted Fate,nagazwitattitude,Evelynn,PayDayAJ21,Vayne,Smammered,Gnar,Helldemmon,Braum,Stickolas


0,1,2,3,4,5,6,7,8,9
Sivir,Froggy Dude,Teemo,Etowng99,Dr. Mundo,UnderpaidSniper,Nidalee,TheOmegaBear09,Ekko,Magnablot


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Dr. Mundo,10,UnderpaidSniper,0.00:1,0,3,0,0,"3,082",0,0,0,0,53,2.4,m,4k Silver 5
Blitzcrank,14,UnderpaidSniper,2.17:1,3,6,10,42,"13,172",0,0,21,3,55,1.4,m,11k Silver 5
Garen,18,UnderpaidSniper,2.33:1,14,6,0,36,"27,279",0,0,12,2,239,6.3,m,16k Silver 5
Blitzcrank,12,UnderpaidSniper,2.60:1,2,5,11,39,"5,659",0,0,17,0,26,1,m,8k Silver 5
Blitzcrank,12,UnderpaidSniper,1.83:1,3,6,8,31,"8,019",0,0,14,3,32,1.1,m,9k Silver 5
Blitzcrank,15,UnderpaidSniper,12.50:1,7,2,18,54,"16,254",0,0,20,2,32,1,m,11k Silver 5
Blitzcrank,14,UnderpaidSniper,10.50:1,5,2,16,42,"11,483",0,0,16,2,31,1.1,m,11k Silver 5
Leona,13,UnderpaidSniper,2.00:1,4,7,10,48,"10,586",0,0,11,1,42,1.3,m,9k Silver 5
Blitzcrank,15,UnderpaidSniper,4.00:1,3,6,21,39,"14,626",0,0,34,3,38,0.9,m,12k Silver 5
Blitzcrank,10,UnderpaidSniper,0.88:1,1,8,6,47,"6,608",0,0,12,0,22,1,m,6k Silver 5


Performace Extrated!
Clone Made!!
#####################   Table of underpaidsniper's Dr. Mundo  #####################


...

#####################   Table of PayDayAJ21's Evelynn  #####################


game_frequency,champ_frequency,win_rate,kda,partcipate,ward_place,ward_kill,cs,tier,top
17,5,0.8,2.791667,38.6,4.0,0.6,2.86,30.0,0.058824


#####################   Table of Smammered's Vayne  #####################


game_frequency,champ_frequency,win_rate,kda,partcipate,ward_place,ward_kill,cs,tier,top
47,8,0.5,2.807018,51.875,14.375,1.0,4.925,30.0,0.025641


#####################   Table of Helldemmon's Gnar  #####################
